### **REVISÃO - Chat Abençoado**

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
# Definindo os caminhos dos arquivos PDF
caminhos = [
    "C:/Users/joao_/Downloads/Portugues-Catolica-AVM-All-Bible.pdf",
    "C:/Users/joao_/Downloads/Catecismo-da-Igreja-Catolica.pdf"
]

# Carregando os documentos PDF
paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())
    
# Dividindo os documentos em pedaços menores
split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n",".", " ", ""]
)

documentos = split.split_documents(paginas)
for i, doc in enumerate(documentos):
    doc.metadata['source'] = doc.metadata['source'].replace('C:/Users/joao_/Downloads/', '')
    doc.metadata['doc_id'] = i

In [4]:
vectorstore = FAISS.from_documents(
    documents=documentos,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
)

In [33]:
# Importando o modelo de chat e criando a cadeia de perguntas e respostas

from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.chains.retrieval_qa.base import RetrievalQA

chat = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

chat_chain = RetrievalQA.from_chain_type(
    llm=chat, 
    retriever=vectorstore.as_retriever(search_type="mmr"),
    chain_type="stuff"
)

In [34]:
from langchain.prompts import ChatPromptTemplate
from langchain.globals import set_debug

prompt_template = ChatPromptTemplate.from_template(
    "Você é um especialista em teologia católica. Responda a pergunta com base nos documentos fornecidos: {query}"
)

In [36]:
pergunta = "O que o Catecismo da Igreja Católica diz sobre a Eucaristia?"
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

O Catecismo da Igreja Católica diz que a Eucaristia é um sacrifício de acção de graças ao Pai, uma bênção pela qual a Igreja exprime o seu reconhecimento a Deus por todos os seus benefícios, por tudo o que Ele fez mediante a criação, a redenção e a santificação. Eucaristia significa, antes de mais, «acção de graças».
Os sacramentos da nova Lei foram instituídos por Cristo e são em número de sete, a saber: o Baptismo, a Confirmação, a Eucaristia, a Penitência, a Unção dos Enfermos, a Ordem e o Matrimónio.
Pela celebração eucarística, unimo-nos desde já à Liturgia do céu e antecipamos a vida eterna, quando «Deus for tudo em todos».
Em síntese, a Eucaristia é o resumo e a súmula da nossa fé: «A nossa maneira de pensar está de acordo com a Eucaristia: e, por sua vez, a Eucaristia confirma a nossa maneira de pensar».
